# TransQTL network multi-omics data simulation and analysis

For some brief background information, please read [this document](https://drive.google.com/file/d/1Y9qRtF45MVuO_9Q6AyV2eerVdzWZI-Dt/view?usp=drive_link) (need to request access to this). ARCHIE method can be found at: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9325868/

## Overview

A starting point for this project is to first try to implement the simulation study design of ARCHIE paper, and apply ARCHIE to this to test if it works. 


- Step 1: Please try to execute the code below and see if you can make it work on your computer. Please try your best to debug it yourselve and fix these codes, if they don't work. Please communicate with us once you can successfully run these code (and tell us what you have changed in the code, if any). 
- Step 2: Examine the output of the code, and try to understand that the code implements the simulation study design described in the ARCHIE paper. To show your understanding, please refactor the 2nd code cell below to various functions each performing some related tasks; and call these functions that you wrote to generate some results. The output should be the same before you refactor, but your code supposed to be much more clear and you should already understand each component of the code, by putting them into logically reasonable components.
- Step 3: Apply a prototype of our new method to this simulation data. Please reach out to us to ask for those codes when you are at the point to move forward.
- Step 4: Compare results from our method and compare that to ARCHIE, as a special case of our method. Define various metrics to argue which version seems to work better? 
- Step 5: We need to implement the simulation study design in Slide 15 to test more general, multi-omics scenarios. If we consider multiple modalities of data, i.e. multiple pairs of (G_r, E_cis_r, E_trans_r), how could we generalize the above simulation studies with respect to the same structure in each modality of multi-omics data? To answer this question please work closely with us. At that point, we will give you more instructions on slack to make sure that you understood the proposed model for data simulation so you can generalize previous simulation to it, with the previous simulation still a special case of this more complex one.
- Step 6: Further generalize the simulation codes above:  for different modality, there may not be the same trans-regulatory structure and with different effect sizes? What are the other parameters to consider to generalize it? Please communicate with us at this step, so we can figure this out together.
- Step 7: Analyze above simulated data using ARCHIE to see how it works -- obviously ARCHIE cannot be applied to multiple modalities so please perform ARCHIE analysis for each simulated modality on its own. 
- Step 8: How do we compare results beteen ARCHIE (just one modality) and our method (more modalities)? A good starting point for you is to first use ARCHIE model to each modality and then combine the results together to do the final evaluation. 

In [1]:
# --------- ARCHIE simulation settings ---------
rm(list = ls())

Generate_Data_one <- function(n, # sample size
                              p, # number of SNPs
                              A_snp_cis, # structure from SNP to cis-gene with elements 0 or 1
                              A_cis_trans, # structure from cis-gene to trans-gene with elements 0 or 1
                              A_trans_trans # structure from trans-gene to trans-gene with elements 0 or 1
){
    # - initial generated data
    data <- list()
    
    # - step 1. generate independent SNPs with MAF in [0.1, 0.4]
    G <- lapply(1:p, function(i) rbinom(n, 2, runif(1, 0.1, 0.4)))
    G <- do.call(cbind, G)
    maf <- colMeans(G)/2
    G = apply(G, 2, scale) # standardize
    data$maf <- maf
    data$G <- G
    
    # step 2. generate cis-gene expressions from genotype 
    E_cis <- matrix(nrow = n, ncol = ncol(A_snp_cis))
    for (i in 1:ncol(A_snp_cis)){
        pos <- which(A_snp_cis[,i] == 1) # which cis-genes are affected by the i-th snp
        beta_i <- rnorm(length(pos), mean = 0.1, sd = sqrt(0.04))
        mu_cis_i <- as.matrix(G[, pos]) %*% as.matrix(beta_i)
        E_cis[,i] <- unlist(lapply(mu_cis_i, function(mu0) rnorm(1, mean = mu0, sd = 1)))
    }
    data$E_cis <- E_cis
    
    # step 3. generate trans-gene expressions from cis-gene expressions
    E_trans <- matrix(0, nrow = n, ncol = ncol(A_trans_trans))
    for (i in 1:ncol(A_trans_trans)){
        pos_cis <- which(A_cis_trans[,i] == 1) # which cis-genes affect the i-th trans-gene
        pos_trans <- which(A_trans_trans[,i] == 1) # which trans-genes affect the i-th trans-gene
        if (length(pos_cis) != 0 | length(pos_trans) != 0){
            temp_cis <- E_cis[, pos_cis, drop = FALSE]
            temp_trans <- E_trans[, pos_trans, drop = FALSE]
            temp <- cbind(temp_cis, temp_trans)
            r <- ncol(temp)
            nu <- 0.1 # first try; need to specific
            gamma <- rnorm(r, mean = 0.1, sd = sqrt(nu/r))
            E_trans[,i] <- temp %*% as.matrix(gamma)
        }
    }
    E_trans <- apply(E_trans, 2, scale)
    data$E_trans <- E_trans
    
    return(data)
}

                                   

We need most of the codes below written as R functions so the main methods comparison benchmark can call these functions easily, using .

In [ ]:
# ------ P.1 Define affection structure
# - 5 snps affect 1 cis-gene
A_snp_cis <- as.matrix(do.call(Matrix::bdiag, lapply(1:8, function(ii) rep(1,5))))

# - specific structure in ARCHIE paper
A_cis_trans <- matrix(0, nrow = 8, ncol = 9)
# cis-gene 1-2 -> E_1 in layer 1; cis-gene 3-4 -> E_2 in layer 1; cis-gene 5-6 -> E_3 in layer 1; cis_gene 7-8 -> E_4 in layer 1
A_cis_trans[c(1,2), 1] <- A_cis_trans[c(3,4), 2] <- A_cis_trans[c(5,6), 3] <- A_cis_trans[c(7,8), 4] <- 1 

A_trans_trans <- matrix(0, nrow = 9, ncol = 9)
# E_1 -> E5 in layer 2; E_2 and E_3 -> E6 in layer 2; E_4 -> E_7 in layer 2
A_trans_trans[1, 5] <- A_trans_trans[c(2,3), 6] <- A_trans_trans[4, 7] <- 1
# E_5 and E_6 -> E_8 in layer 3; E_6 and E_7 -> E_9 in layer 3
A_trans_trans[c(5,6), 8] <- A_trans_trans[c(6,7), 9] <- 1

# ------- P.2 Generate data for estimating Sigma_GE
n <- 1000
p <- 40
set.seed(2222)
data <- Generate_Data_one(n, p, A_snp_cis, A_cis_trans, A_trans_trans)
# calcualte z-score between snps and trans-genes
Z <- matrix(nrow = p, ncol = ncol(data$E_trans))
for (i in 1:p){
    for (j in 1:ncol(data$E_trans)){
        res = susieR::univariate_regression(data$G[,i], data$E_trans[,j])
        Z[i,j] <- res$betahat / res$sebetahat
    }
}
pheatmap::pheatmap(Z, cluster_rows=FALSE, cluster_cols=FALSE)
Sigma_GE <- Z / sqrt(n * 2 * data$maf * (1-data$maf))
pheatmap::pheatmap(Sigma_GE, cluster_rows=FALSE, cluster_cols=FALSE)

# -------- P.3 Generate independent data for estimating Sigma_EE
n_ind <- 700
data_ind <- Generate_Data_one(n_ind, p, A_snp_cis, A_cis_trans, A_trans_trans)
Sigma_EE <- cor(data_ind$E_trans) # correlation in paper but covariance in supplementary
pheatmap::pheatmap(Sigma_EE, cluster_rows=FALSE, cluster_cols=FALSE)

# -------- P.4 Estimate Sigma_GG, in ARCHIE, it is identity matrix anyway
Sigma_GG <- diag(apply(data$G, 2, var))

# -------- P.5 run ARCHIE
setwd("~/Library/CloudStorage/GoogleDrive-xwcaochn@gmail.com/My Drive/transQTL/ARCHIE/Simulation")
source("main_ARCHIE.R")
source("helper.R")
A1 <- archie_work(Sigma_GE,Sigma_GG,Sigma_EE)
us <- A1$us
vs <- A1$vs

# --- But we saw the issue for ARCHIE code is that they calculate W as the following formula
W <- Sigma_GG%*%Sigma_GE%*%Sigma_EE
pheatmap::pheatmap(W, cluster_rows=FALSE, cluster_cols=FALSE)

# --- In their paper, they define in the different way: W = Sigma_GG^(-1/2) %*% Sigma_GE $*$ Sigma_EE^(-1/2)
Sigma_GE_define <- t(data$G) %*% data$E_trans
Sigma_EE_define <- cor(data_ind$E_trans) # correlation in paper but covariance in supplementary
Sigma_GG_define <- Sigma_GG

# Since Sigma_GG is identity matrix
Sigma_GG_inv_sqrt <- Sigma_GG
# hard part is how to take inverse square root of Sigma_EE_define
eigen_EE <- eigen(Sigma_EE_define)
eigen_EE$values
# - way 1: add regulatory penalty 0.001
eigen_EE <- eigen((1-0.001)*Sigma_EE_define+0.001*diag(1, nrow = nrow(Sigma_EE_define)))
eigen_EE$values
Sigma_EE_inv_sqrt <- t(eigen_EE$vectors) %*% diag(1/sqrt(eigen_EE$values)) %*% eigen_EE$vectors

W_way1 <- Sigma_GG_inv_sqrt%*%Sigma_GE_define%*%Sigma_EE_inv_sqrt
pheatmap::pheatmap(W_way1, cluster_rows=FALSE, cluster_cols=FALSE)
# - try ARCHIE by corrected W
source("archie_work_correctW.R")
A2 <- archie_work_correctW(W_way1)
us2 <- A2$us
vs2 <- A2$vs

# - results for us2 and vs2 are not correct, then the issue may be exist in Sigma_GE, we may need correlation of G and E_trans
Sigma_GE_define <- cor(data$G, data$E_trans)
W_way1 <- Sigma_GG_inv_sqrt%*%Sigma_GE_define%*%Sigma_EE_inv_sqrt
pheatmap::pheatmap(W_way1, cluster_rows=FALSE, cluster_cols=FALSE)
# - try again ARCHIE by corrected W
source("archie_work_correctW.R")
A2 <- archie_work_correctW(W_way1)
us2 <- A2$us
vs2 <- A2$vs
# - issue now is that W in ARCHIE code is not the same as W_way1 introduced in their paper

# - way 2: approximate inverse by removing small eigen values
eigen_EE <- eigen(Sigma_EE_define)
eigen_EE$values 
L <- which(cumsum(eigen_EE$values) / sum(eigen_EE$values) > 0.999)[1]
Sigma_EE_inv_sqrt <- eigen_EE$vectors[,1:L] %*% diag(1/sqrt(eigen_EE$values[1:L])) %*% t(eigen_EE$vectors[,1:L])

# - designed
W_way2 <- Sigma_GG_inv_sqrt%*%Sigma_GE_define%*%Sigma_EE_inv_sqrt
pheatmap::pheatmap(W_way2, cluster_rows=FALSE, cluster_cols=FALSE)
# - try ARCHIE by corrected W
source("archie_work_correctW.R")
A3 <- archie_work_correctW(W_way2)
us3 <- A3$us
vs3 <- A3$vs